# Basic recommendation syste, based on weight score(content based)
# Recommendation based on weighted_average
# Recommendation based on scaled weighted average and popularity score(Priority-50% for both)
https://medium.com/@developeraritro/building-a-recommendation-system-using-weighted-hybrid-technique-75598b6be8ed
https://codeburst.io/explanation-of-recommender-systems-in-information-retrieval-13077e1d916c

In [1]:
import numpy as np #libraries
import pandas as pd #pandas for data frames


In [2]:
credit=pd.read_csv("tmdb_5000_credits.csv") #load 5000 movie data sets where we have ID,TITTLE,CREW,CAST
credit.head(5) #displaying data

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
movie=pd.read_csv("tmdb_5000_movies.csv") #load 5000 movie data where we have have further more information regarding movie
movie.head(5) #ID,TITTLE are same as above datasets

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credit.isnull().values.any() #checking NULL in Credit

False

In [5]:
movie.isnull().values.any() #checking NULL in movie data ->if present then ignore it  

True

In [6]:
credit.info() #info of credit 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [7]:
movie.info() #info of movie

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [8]:
movie.columns #coluymns name of movie

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [9]:
#droping inwanted column which are not in used and storing data frame into new (name it NEW) 
new=movie.drop(['title','genres','keywords','budget','homepage','original_language','original_title', 'overview', 'production_companies','production_countries', 'release_date', 'revenue', 'runtime','spoken_languages', 'status', 'tagline'],axis=1)

In [10]:
new.head(5) #new data frame 

,id,popularity,vote_average,vote_count
0,19995,150.437577,7.2,11800
1,285,139.082615,6.9,4500
2,206647,107.376788,6.3,4466
3,49026,112.312950,7.6,9106
4,49529,43.926995,6.1,2124


In [11]:
credidmovieidrename=credit.rename(index=str, columns={"movie_id":"id"}) #renaming movie-ID into ID in credit data frame and stroring into new


In [12]:
credidmovieidrename.head(2) #new credit data frame

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [13]:
newmoviedata=new.merge(credidmovieidrename,on="id") #merging newcredit data frame and our new movie data frame on bases of ID

In [14]:
newmoviedata.head(2)

,id,popularity,vote_average,vote_count,title,cast,crew
0,19995,150.437577,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,139.082615,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [15]:
redaytowork=newmoviedata.drop(["title","cast","crew"],axis=1) #droping TITTLE,CAST,CREW and storing into readytowork data
redaytowork.head(1) #now we have ID,POPUlARITY<VOTE_AVERAGE,VOTE_COUNT

,id,popularity,vote_average,vote_count
0,19995,150.437577,7.2,11800


In [16]:
redaytowork.info() # all features are in numeric format , no object is present

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   popularity    4803 non-null   float64
 2   vote_average  4803 non-null   float64
 3   vote_count    4803 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 187.6 KB


# now we have to calculate weight of each and every id 

In [17]:
print("weight=((votecount*voteaverage)+meanofvoteaverage*votecountquantile)/(votecount+votecountqunatile")
print("quntile=====70%")

weight=((votecount*voteaverage)+meanofvoteaverage*votecountquantile)/(votecount+votecountqunatile
quntile=====70%


In [18]:
v=redaytowork["vote_count"] #storing all votecount into V
v.head(2)

0    11800
1     4500
Name: vote_count, dtype: int64

In [19]:
r=redaytowork["vote_average"] #storing vote average of each id in R
r.head(3)

0    7.2
1    6.9
2    6.3
Name: vote_average, dtype: float64

In [20]:
meanofr=redaytowork["vote_average"].mean() #taking mean of variable R and storing in MEANOFR
meanofr

6.092171559442011

In [21]:
qunatilev=redaytowork["vote_count"].quantile(0.70) #70 % of VOTECOUNT 
qunatilev

581.0

# formula for weight =((r  v)+(meanofr  qunatilev))/(v+qunatilev)

In [22]:
redaytowork["weight"]=((r*v)+(meanofr*qunatilev))/(v+qunatilev)
redaytowork.head(3) #creating differnt column and storing all weights

,id,popularity,vote_average,vote_count,weight
0,19995,150.437577,7.2,11800,7.148013
1,285,139.082615,6.9,4500,6.807627
2,206647,107.376788,6.3,4466,6.276075


# weight sort in order

In [23]:
weightsort=redaytowork.sort_values("weight",ascending=False)
weightsort.head(5) #sorting weights from higher to lower

,id,popularity,vote_average,vote_count,weight
1881,278,136.747729,8.5,8205,8.340775
3337,238,143.659698,8.4,5893,8.192887
662,550,146.757391,8.3,9413,8.171648
3232,680,121.463076,8.3,8428,8.157615
65,155,187.322927,8.2,12002,8.102674


# popularity sort in order

In [24]:
popularitysort=redaytowork.sort_values("popularity",ascending=False)
popularitysort.head(5) #sorting popularity from higher to lower

,id,popularity,vote_average,vote_count,weight
546,211672,875.581305,6.4,4571,6.365286
95,157336,724.247784,8.1,10867,7.998100
788,293660,514.569956,7.4,10995,7.334360
94,118340,481.098624,7.9,9742,7.798252
127,76341,434.278564,7.2,9427,7.135687


# normalization 0-1

In [25]:
from sklearn.preprocessing import MinMaxScaler #normalizing our POPULAREITUY and WEIGHT feature for better result
scaling=MinMaxScaler()
scaled=scaling.fit_transform(weightsort[["popularity","weight"]])
normalize=pd.DataFrame(scaled,columns=["weight","popularity"])
normalize.head(2) #NORMALIZE data

,weight,popularity
0,0.156179,1.000000
1,0.164074,0.959622


In [26]:
weightsort[["normalize weight","normalize popularity"]]=normalize #storinhg NORMALIZE data into our dataframe 

In [27]:
weightsort.head(3) #as we can see that (normalize weight	normalize popularity)

,id,popularity,vote_average,vote_count,weight,normalize weight,normalize popularity
1881,278,136.747729,8.5,8205,8.340775,0.020092,0.404155
3337,238,143.659698,8.4,5893,8.192887,0.003182,0.377825
662,550,146.757391,8.3,9413,8.171648,0.030701,0.519508


# weight is also depent on recommendation as well as popularity of movie
# so we take both witr some criteria
# so here we taking 50 % of weight and 50 % of popularity of movie for recommendation

In [28]:
print("50 percentage popularuty 50 percentage weight")

50 percentage popularuty 50 percentage weight


# creating a different column where we have 50%-50% of both(weight and popularity) 

In [29]:
score=weightsort["normalize popularity"]*.5+weightsort["normalize weight"]*.5
score.head(3)

1881    0.212124
3337    0.190503
662     0.275105
dtype: float64

In [30]:
weightsort["score"]=score #added score column in data frame
weightsort.head(5)

,id,popularity,vote_average,vote_count,weight,normalize weight,normalize popularity,score
1881,278,136.747729,8.5,8205,8.340775,0.020092,0.404155,0.212124
3337,238,143.659698,8.4,5893,8.192887,0.003182,0.377825,0.190503
662,550,146.757391,8.3,9413,8.171648,0.030701,0.519508,0.275105
3232,680,121.463076,8.3,8428,8.157615,0.005801,0.379687,0.192744
65,155,187.322927,8.2,12002,8.102674,0.067903,0.796938,0.432421


# final data frame with score features

In [31]:
finalset=weightsort.sort_values(["score"],ascending=False)
finalset.head(5)

,id,popularity,vote_average,vote_count,weight,normalize weight,normalize popularity,score
11,10764,107.928811,6.1,2965,6.098717,0.827162,0.906439,0.866801
1071,232672,44.120328,6.6,1006,6.414084,1.000000,0.460630,0.730315
29,37724,93.004993,6.9,7604,6.842658,0.549462,0.851874,0.700668
135,7978,21.214571,5.5,549,5.804471,0.587690,0.725217,0.656453
210,415,50.073575,4.2,1418,4.749951,0.495989,0.670973,0.583481


In [32]:
finalset

,id,popularity,vote_average,vote_count,weight,normalize weight,normalize popularity,score
11,10764,107.928811,6.1,2965,6.098717,0.827162,0.906439,0.866801
1071,232672,44.120328,6.6,1006,6.414084,1.000000,0.460630,0.730315
29,37724,93.004993,6.9,7604,6.842658,0.549462,0.851874,0.700668
135,7978,21.214571,5.5,549,5.804471,0.587690,0.725217,0.656453
210,415,50.073575,4.2,1418,4.749951,0.495989,0.670973,0.583481
...,...,...,...,...,...,...,...,...
4797,67238,0.022173,7.5,2,6.097001,0.006926,0.102239,0.054583
4800,231617,1.444476,7.0,6,6.101451,0.057189,0.019593,0.038391
4799,72766,0.642552,5.9,5,6.090532,0.027595,0.039279,0.033437
4801,126186,0.857008,5.7,7,6.087503,0.043544,0.017935,0.030740
